#### Creación base de datos - SQL
Este notebook se va a utilizar para crear y rellenar la base de datos con las tablas previamente trabajadas.

In [1]:
import mysql.connector as conn
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
with open('contraseñas.txt') as file:
    mypasswd = file.read()

In [3]:
# Nos conectamos al servidor 
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd=f'{mypasswd}'
                        )

cursor  = conexion.cursor()

cursor

In [4]:
cursor.execute('drop database if exists transito')

cursor.execute('create database transito;')

In [5]:
conexion = conn.connect(host='localhost',
                        user='root',
                        passwd=f'{mypasswd}',
                        database='transito'
                        )

cursor  = conexion.cursor()

In [6]:
c = cursor.execute
c

<bound method CMySQLCursor.execute of <mysql.connector.cursor_cext.CMySQLCursor object at 0x00000278C956EEC0>>

#### Insertamos tablas en el database

In [38]:
# Leemos los 4 archivos csv que nos interesan
df_transit_clean = pd.read_csv('../data/transit_clean.csv')
df_puertos = pd.read_csv('../data/puertos.csv')
df_transit_plus_weather = pd.read_csv('../data/transit_plus_weather.csv')
df_otle_gt = pd.read_csv('../data/otle_gt.csv')

#### Transit clean

In [8]:
df_transit_clean.head()

,transit_id,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0
1,2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0
2,3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0
3,4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0
4,5,HYPATIA DE ALEJANDRIA,BARCELONA,BARCELONA,01/06/2023 05:19,01/06/2023 11:35,MUELLE DE LA 2 ALIN. DEL DIQUE DEL OESTE (del ...,BALEARIA EUROLINEAS MARITIMAS S.A.U.,28.658,2181.0,CHIPRE,187.0,7.0


In [10]:
c('drop table if exists transit_clean;')

query ='''

create table transit_clean(
    transit_id int primary key not null,
    barco varchar(45),
    origen varchar(45),
    destino	varchar(45),
    llegada	varchar(45),
    salida varchar(45),
    alineacion varchar(100),
    consignatorio varchar(100),
    gt float,
    escala float,
    bandera varchar(45),
    eslora float,
    calado float
    );
'''

c(query)

In [11]:
# Relleno datos
nombre_tabla = 'transit_clean'
nombre_columnas = ','.join(df_transit_clean.columns) # creo una string

#bucle por filas
for i in range(df_transit_clean.shape[0]):
    valores = tuple(df_transit_clean.iloc[i].values)

    insert_query = f'insert into {nombre_tabla} ({nombre_columnas}) values {valores}'

    cursor.execute(insert_query)


In [12]:
insert_query

"insert into transit_clean (transit_id,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado) values (1049, 'ELEANOR ROOSEVELT', 'IBIZA', 'IBIZA', '31/08/2023 22:25', '01/09/2023 07:50', 'MUELLE PARAIRES (del 11 al 1)', 'BALEARIA EUROLINEAS MARITIMAS S.A.U.', 12.262, 3106.0, 'CHIPRE', 123.0, 4.0)"

In [13]:
# Comprobamos

cursor.execute('select * from transit_clean limit 2;')

for x in cursor:
    print(x)

(1, 'VALIANT LADY', 'CANNES', 'IBIZA', '31/05/2023 17:44', '01/06/2023 18:15', 'ALIN.NORTE DE LA PLATAFORMA (del 22 al 2)', 'ESTELA SHIPPING PALMA SA', 108.192, 192.0, 'BAHAMAS', 277.0, 8.0)
(2, 'ELEANOR ROOSEVELT', 'IBIZA', 'IBIZA', '31/05/2023 22:28', '01/06/2023 07:51', 'MUELLE PARAIRES (del 11 al 2)', 'BALEARIA EUROLINEAS MARITIMAS S.A.U.', 12.262, 2066.0, 'CHIPRE', 123.0, 4.0)


In [14]:
# Comprobamos tamaño
# check 

cursor.execute('select count(*) from transit_clean;')

for x in cursor:
    print(x)

(1049,)


#### Puertos

In [19]:
df_puertos = df_puertos.rename(columns={'transit_coords_id': 'puertos_id'})
df_puertos.head()

,puertos_id,Puerto,Latitud,Longitud
0,0,IBIZA,38.912653,1.442283
1,1,BARCELONA,41.346387,2.165532
2,2,Valencia,39.467328,-0.390288
3,3,"MAHON, MENORCA",39.904400,4.247600
4,4,SAGUNTO,39.783600,-0.210800


In [20]:
c('drop table if exists puertos;')

query ='''

create table puertos(
    puertos_id int primary key not null,
    Puerto varchar(100),
    Latitud float,
    Longitud float
    );
'''

c(query)

In [21]:
# Relleno datos
nombre_tabla = 'puertos'
nombre_columnas = ','.join(df_puertos.columns) # creo una string

#bucle por filas
for i in range(df_puertos.shape[0]):
    valores = tuple(df_puertos.iloc[i].values)

    insert_query = f'insert into {nombre_tabla} ({nombre_columnas}) values {valores}'

    cursor.execute(insert_query)

In [22]:
insert_query

"insert into puertos (puertos_id,Puerto,Latitud,Longitud) values (42, 'PORT EVERGLADES', 26.166667, -80.166667)"

In [23]:
# Comprobamos

cursor.execute('select * from puertos limit 2;')

for x in cursor:
    print(x)

(0, 'IBIZA', 38.9127, 1.44228)
(1, 'BARCELONA', 41.3464, 2.16553)


In [24]:
# Comprobamos tamaño
# check 

cursor.execute('select count(*) from puertos;')

for x in cursor:
    print(x)

(43,)


#### Transit + Weather

In [25]:
df_transit_plus_weather = df_transit_plus_weather.rename(columns={'Unnamed: 0': 'transit_id'})
df_transit_plus_weather.head()

,transit_id,barco,puerto_llegada,LatitudPuertoLlegada,LongitudPuertoLlegada,llegada_fecha,llegada_hora,salida_fecha,salida_hora,daily_rain_sum,daily_wind_speed_10m_max
0,0,VALIANT LADY,IBIZA,38.912653,1.442283,2023-05-31,17:44,2023-06-01,18:15,0.0,8.209263
1,1,ELEANOR ROOSEVELT,IBIZA,38.912653,1.442283,2023-05-31,22:28,2023-06-01,07:51,0.0,8.209263
2,2,TENACIA,BARCELONA,41.346387,2.165532,2023-06-01,04:23,2023-06-01,09:59,2.7,14.336861
3,3,HEDY LAMARR,Valencia,39.467328,-0.390288,2023-06-01,04:53,2023-06-01,10:21,1.2,18.345877
4,4,HYPATIA DE ALEJANDRIA,BARCELONA,41.346387,2.165532,2023-06-01,05:19,2023-06-01,11:35,2.7,14.336861


In [26]:
c('drop table if exists transit_plus_weather;')

query ='''

create table transit_plus_weather(
    transit_id int primary key not null,
    barco varchar(45),
    puerto_llegada varchar(45),
    LatitudPuertoLlegada float,
    LongitudPuertoLlegada float,
    llegada_fecha varchar(45),
    llegada_hora varchar(45),
    salida_fecha varchar(45),
    salida_hora varchar(45),
    daily_rain_sum float,
    daily_wind_speed_10m_max float
    );
'''

c(query)

In [27]:
# Relleno datos
nombre_tabla = 'transit_plus_weather'
nombre_columnas = ','.join(df_transit_plus_weather.columns) # creo una string

#bucle por filas
for i in range(df_transit_plus_weather.shape[0]):
    valores = tuple(df_transit_plus_weather.iloc[i].values)

    insert_query = f'insert into {nombre_tabla} ({nombre_columnas}) values {valores}'

    cursor.execute(insert_query)

In [28]:
insert_query

"insert into transit_plus_weather (transit_id,barco,puerto_llegada,LatitudPuertoLlegada,LongitudPuertoLlegada,llegada_fecha,llegada_hora,salida_fecha,salida_hora,daily_rain_sum,daily_wind_speed_10m_max) values (1048, 'ELEANOR ROOSEVELT', 'IBIZA', 38.912653, 1.442283, '2023-08-31', '22:25', '2023-09-01', '07:50', 0.0, 16.279802)"

In [29]:
# Comprobamos

cursor.execute('select * from transit_plus_weather limit 2;')

for x in cursor:
    print(x)

(0, 'VALIANT LADY', 'IBIZA', 38.9127, 1.44228, '2023-05-31', '17:44', '2023-06-01', '18:15', 0.0, 8.20926)
(1, 'ELEANOR ROOSEVELT', 'IBIZA', 38.9127, 1.44228, '2023-05-31', '22:28', '2023-06-01', '07:51', 0.0, 8.20926)


In [30]:
# Comprobamos tamaño
# check 

cursor.execute('select count(*) from transit_plus_weather;')

for x in cursor:
    print(x)

(1049,)


#### OTLE GT 

In [39]:
df_otle_gt = df_otle_gt.rename(columns={'Unnamed: 0': 'otle_id'})
df_otle_gt.head()

,otle_id,autoridad_portuaria,tipo_buque,procedencia,year,num_buques,gt
0,0,Autoridad Portuaria A Coruña,Tanqueros,Nacionales,2019,3.0,54.468
1,1,Autoridad Portuaria A Coruña,Tanqueros,Nacionales,2020,4.0,21.696
2,2,Autoridad Portuaria A Coruña,Tanqueros,Nacionales,2021,1.0,5.424
3,3,Autoridad Portuaria A Coruña,Tanqueros,Extranjeros,2019,424.0,9.472.798
4,4,Autoridad Portuaria A Coruña,Tanqueros,Extranjeros,2020,287.0,7.244.978


In [46]:
# Voy a tener probelmas con los puntos de la columna 'gt' así que voy a quitar los puntos 
df_otle_gt['gt'] = df_otle_gt['gt'].str.replace('.', '')
df_otle_gt.head()

,otle_id,autoridad_portuaria,tipo_buque,procedencia,year,num_buques,gt
0,0,Autoridad Portuaria A Coruña,Tanqueros,Nacionales,2019,3.0,54468
1,1,Autoridad Portuaria A Coruña,Tanqueros,Nacionales,2020,4.0,21696
2,2,Autoridad Portuaria A Coruña,Tanqueros,Nacionales,2021,1.0,5424
3,3,Autoridad Portuaria A Coruña,Tanqueros,Extranjeros,2019,424.0,9472798
4,4,Autoridad Portuaria A Coruña,Tanqueros,Extranjeros,2020,287.0,7244978


In [47]:
c('drop table if exists otle_gt;')

query ='''

create table otle_gt(
    otle_id int primary key not null,
    autoridad_portuaria	varchar(100),
    tipo_buque varchar(45),
    procedencia varchar(45),
    year int,
    num_buques int,
    gt float
    );
'''

c(query)

In [48]:
# Relleno datos
nombre_tabla = 'otle_gt'
nombre_columnas = ','.join(df_otle_gt.columns) # creo una string

#bucle por filas
for i in range(df_otle_gt.shape[0]):
    valores = tuple(df_otle_gt.iloc[i].values)

    insert_query = f'insert into {nombre_tabla} ({nombre_columnas}) values {valores}'

    cursor.execute(insert_query)

In [49]:
insert_query

"insert into otle_gt (otle_id,autoridad_portuaria,tipo_buque,procedencia,year,num_buques,gt) values (1175, 'Autoridad Portuaria Vilagarcía', 'Solo de pasaje', 'Extranjeros', 2021, 4.0, '153468')"

In [50]:
# Comprobamos

cursor.execute('select * from otle_gt limit 2;')

for x in cursor:
    print(x)

(0, 'Autoridad Portuaria A Coruña', 'Tanqueros', 'Nacionales', 2019, 3, 54468.0)
(1, 'Autoridad Portuaria A Coruña', 'Tanqueros', 'Nacionales', 2020, 4, 21696.0)


In [51]:
# Comprobamos tamaño
# check 

cursor.execute('select count(*) from otle_gt;')

for x in cursor:
    print(x)

(1113,)


In [52]:
cursor.execute('show tables;')

for e in cursor:
    print(e)

('otle_gt',)
('puertos',)
('transit_clean',)
('transit_plus_weather',)
('trnasit_clean',)


* Me falta borrar una tabla que hice por error con una errata

In [54]:
table_name = 'trnasit_clean'
try:
    cursor.execute(f'DROP TABLE {table_name}')
    print('Table was dropped succesfully')
except:
    print('Error, table not found or already dropped beforehand.')


Table was dropped succesfully


In [55]:
cursor.execute('show tables;')

for e in cursor:
    print(e)

('otle_gt',)
('puertos',)
('transit_clean',)
('transit_plus_weather',)
